In [1]:
!pip -q install transformers accelerate sentencepiece pypdf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 5.4 MB/s eta 0:00:00


In [9]:
import re, math, textwrap, torch
from typing import List, Dict, Tuple
from dataclasses import dataclass
from transformers import (
    AutoTokenizer, AutoModelForQuestionAnswering, pipeline,
    AutoModelForSeq2SeqLM
)
from pypdf import PdfReader

DEVICE = 0 if torch.cuda.is_available() else -1
print("Device:", "GPU" if DEVICE==0 else "CPU")

# --- (A) 한국어 추출형 QA 파이프라인: KoELECTRA (KorQuAD 파인튜닝) ---
qa_tok = AutoTokenizer.from_pretrained("monologg/koelectra-small-v3-discriminator")
qa_model = AutoModelForQuestionAnswering.from_pretrained(
    "monologg/koelectra-small-v3-finetuned-korquad"
)
qa = pipeline("question-answering", model=qa_model, tokenizer=qa_tok, device=DEVICE)

# --- (B) 한국어 요약 파이프라인: KoBART 요약 ---
# KoBART는 fast 토크나이저 이슈가 있을 수 있어 use_fast=False로 로드
sum_name = "gogamza/kobart-summarization"
# Changed use_fast=False to use_fast=True
sum_tok = AutoTokenizer.from_pretrained(sum_name, use_fast=True)
sum_model = AutoModelForSeq2SeqLM.from_pretrained(sum_name)
summarizer = pipeline("summarization", model=sum_model, tokenizer=sum_tok, device=DEVICE)

Device: CPU


Device set to use cpu
You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.
You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.
You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.
Device set to use cpu


토큰 단위 슬라이딩 윈도

In [10]:
@dataclass
class Chunk:
    idx: int
    text: str
    start_char: int
    end_char: int
    token_len: int

def chunk_by_tokens(
    text: str,
    tokenizer,
    max_tokens: int = 384,     # 청크 크기(토큰)
    stride_tokens: int = 64     # 겹침(오버랩) 크기(토큰)
) -> List[Chunk]:
    """
    문서를 토큰 단위로 슬라이딩 윈도 청킹합니다.
    - max_tokens : 한 청크의 최대 토큰 수
    - stride_tokens : 다음 청크로 이동할 때 겹치는 토큰 수(문맥 보존)
    반환: 청크 리스트(텍스트와 원문 내 문자 위치 포함)
    """
    assert stride_tokens < max_tokens, "stride_tokens < max_tokens 여야 합니다."

    # 토큰화(오프셋 매핑으로 원문 문자 위치 역추적)
    enc = tokenizer(
        text, return_offsets_mapping=True, add_special_tokens=False
    )
    input_ids = enc["input_ids"]
    offsets = enc["offset_mapping"]

    chunks: List[Chunk] = []
    start_tok = 0
    idx = 0
    while start_tok < len(input_ids):
        end_tok = min(start_tok + max_tokens, len(input_ids))
        # 토큰 → 문자 위치
        start_char = offsets[start_tok][0] if start_tok < len(offsets) else 0
        end_char = offsets[end_tok-1][1] if end_tok-1 < len(offsets) else len(text)
        chunk_text = text[start_char:end_char]

        chunks.append(
            Chunk(
                idx=idx,
                text=chunk_text,
                start_char=start_char,
                end_char=end_char,
                token_len=(end_tok - start_tok),
            )
        )
        idx += 1
        if end_tok == len(input_ids):
            break
        # 슬라이딩: (현재 시작 + max_tokens - stride) 지점으로 이동
        start_tok = end_tok - stride_tokens

    return chunks


PDF 텍스트 추출 함수

In [11]:
def read_pdf_text(pdf_path: str, max_pages: int = None) -> str:
    reader = PdfReader(pdf_path)
    pages = reader.pages[:max_pages] if max_pages else reader.pages
    texts = []
    for p in pages:
        t = p.extract_text() or ""
        texts.append(t)
    return "\n".join(texts)


In [12]:
# ① 직접 텍스트를 넣어 데모
demo_text = """
소나기는 1959년에 발표된 황순원의 단편소설이다. 한 소년과 소녀의 풋풋한 감정이 여름 들판과 소나기라는 배경 속에서 그려진다.
작품은 강렬한 자연의 이미지와 사소한 사건들을 통해 성장의 순간을 섬세하게 포착한다.
특히 갑작스런 소나기와 그 후의 정적은 인물들의 감정을 비유적으로 드러내는 장치로 기능한다.
이 작품은 한국 단편소설의 고전으로 평가받는다.
""" * 20  # 길이를 늘리기 위해 반복

# ② PDF로 실습하려면 아래 주석 해제 후 파일 경로 지정
# from google.colab import files
# uploaded = files.upload()  # PDF 업로드
# pdf_path = list(uploaded.keys())[0]
# demo_text = read_pdf_text(pdf_path, max_pages=10)


청킹 시각화

In [13]:
# 토큰 기준 청킹
chunks = chunk_by_tokens(
    demo_text,
    tokenizer=qa_tok,         # QA 토크나이저 기준으로 청킹하면 좋음
    max_tokens=384,
    stride_tokens=64
)

print(f"총 청크 수: {len(chunks)}  |  첫 3개 미리보기")
for c in chunks[:3]:
    preview = textwrap.shorten(c.text.replace("\n", " "), width=120)
    print(f"[{c.idx:02d}] tokens={c.token_len:3d}  chars=({c.start_char}-{c.end_char})  {preview}")


Token indices sequence length is longer than the specified maximum sequence length for this model (1980 > 512). Running this sequence through the model will result in indexing errors


총 청크 수: 6  |  첫 3개 미리보기
[00] tokens=384  chars=(1-782)  소나기는 1959년에 발표된 황순원의 단편소설이다. 한 소년과 소녀의 풋풋한 감정이 여름 들판과 소나기라는 배경 속에서 그려진다. 작품은 강렬한 자연의 이미지와 사소한 사건들을 통해 성장의 순간을 섬세하게 [...]
[01] tokens=384  chars=(645-1428)  한 감정이 여름 들판과 소나기라는 배경 속에서 그려진다. 작품은 강렬한 자연의 이미지와 사소한 사건들을 통해 성장의 순간을 섬세하게 포착한다. 특히 갑작스런 소나기와 그 후의 정적은 인물들의 감정을 [...]
[02] tokens=384  chars=(1297-2076)  사소한 사건들을 통해 성장의 순간을 섬세하게 포착한다. 특히 갑작스런 소나기와 그 후의 정적은 인물들의 감정을 비유적으로 드러내는 장치로 기능한다. 이 작품은 한국 단편소설의 고전으로 평가받는다. 소나기는 [...]


청킹 기반 한국어 QA

In [14]:
def qa_over_chunks(question: str, chunks: List[Chunk], top_k: int = 3):
    scored = []
    for c in chunks:
        try:
            out = qa({"question": question, "context": c.text})
            scored.append((out["score"], out["answer"], c))
        except Exception as e:
            # 청크가 너무 짧거나 토큰 이슈가 있을 수 있음 → 스킵
            continue
    # score 내림차순
    scored.sort(key=lambda x: x[0], reverse=True)
    return scored[:top_k]

question = "이 작품의 작가는 누구인가?"
top_answers = qa_over_chunks(question, chunks, top_k=3)

print(f"[질문] {question}\n")
for rank, (score, ans, c) in enumerate(top_answers, 1):
    prev = textwrap.shorten(c.text.replace("\n", " "), width=80)
    print(f"#{rank}  score={score:.4f}  answer=「{ans}」  (chunk={c.idx}, tokens={c.token_len})")
    print(f"    └ context: {prev}")


/usr/local/lib/python3.12/dist-packages/transformers/pipelines/question_answering.py:395: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(


[질문] 이 작품의 작가는 누구인가?

#1  score=1.2905  answer=「황순원의」  (chunk=5, tokens=380)
    └ context: 한 소년과 소녀의 풋풋한 감정이 여름 들판과 소나기라는 배경 속에서 그려진다. 작품은 강렬한 자연의 이미지와 사소한 사건들을 통해 [...]
#2  score=1.1119  answer=「황순원의」  (chunk=4, tokens=384)
    └ context: 고전으로 평가받는다. 소나기는 1959년에 발표된 황순원의 단편소설이다. 한 소년과 소녀의 풋풋한 감정이 여름 들판과 소나기라는 배경 [...]
#3  score=1.0679  answer=「황순원의」  (chunk=0, tokens=384)
    └ context: 소나기는 1959년에 발표된 황순원의 단편소설이다. 한 소년과 소녀의 풋풋한 감정이 여름 들판과 소나기라는 배경 속에서 그려진다. [...]


청킹 기반 요약 (Map-Reduce)

In [15]:
def summarize_long_text(
    text: str,
    tokenizer,
    chunk_tokens: int = 600,   # KoBART 입력 길이 감안(여유 잡기)
    stride_tokens: int = 100,
    map_max_new_tokens: int = 120,
    reduce_max_new_tokens: int = 150,
) -> Dict[str, str]:
    # 1) 청킹 (요약 기준으로 토크나이저 바꿔도 OK)
    sum_chunks = chunk_by_tokens(text, summarizer.tokenizer, chunk_tokens, stride_tokens)

    # 2) Map 단계: 청크별 요약
    partials = []
    for c in sum_chunks:
        # 너무 짧은 청크는 스킵
        if c.token_len < 50:
            continue
        try:
            s = summarizer(
                c.text,
                max_length=map_max_new_tokens,
                min_length=min(50, map_max_new_tokens-10),
                do_sample=False
            )[0]["summary_text"].strip()
            partials.append(s)
        except Exception:
            continue

    # 3) Reduce 단계: 부분 요약들을 하나로 합쳐 다시 요약
    reduce_input = "\n".join(partials)
    final = summarizer(
        reduce_input,
        max_length=reduce_max_new_tokens,
        min_length=min(70, reduce_max_new_tokens-20),
        do_sample=False
    )[0]["summary_text"].strip()

    return {"map_summaries": partials, "final_summary": final}

sum_out = summarize_long_text(demo_text, sum_tok)
print("▶ 청크 요약 개수:", len(sum_out["map_summaries"]))
print("\n[최종 요약]\n", textwrap.fill(sum_out["final_summary"], width=90))


Both `max_new_tokens` (=256) and `max_length`(=120) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=120) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=120) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=120) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


▶ 청크 요약 개수: 4

[최종 요약]
 1959년에 발표된 황순원의 단편소설인 한 소년과 소녀의 풋풋한 감정이 여름 들판과 소나기라는 배경 속에서 그려져 있고, 한 소년과 소녀의 풋풋한 감정이 여름
들판과 소나기라는 배경 속에서 그려져 있다. 한 소년과 소녀의 풋풋한 감정이 여름 들판과 소나기라는 배경 속에서 그려져 있다. 한 소년과 소녀의 풋풋한 감정이
여름 들판과 소나기라는 배경 속에서 그려져 있다. 한 소년과 소녀의 풋풋한 감정이 여름 들판과 소나기라는 배경 속에서 그려져 있다. 한 소년과 소녀의 풋풋한
감정이 여름 들판과 소나기라는 배경 속에서 그려졌을작품은 강렬한 자연의 이미지와 사소한 사건들을 통해 성장의 순간을 섬세하게 포착하고, 성장의 순간을 섬세하게
포착하고, 성장의 순간을 섬세하게 정적은 인물들의 감정을 비유적으로 드러내는 장치로 기능하였다.이 작품은 한국 단편소설의 고전으로 평가되고, 황금 황금소나기는
1959년에 발표된 황순원의 단편소설인 한 소년과 소녀의 풋풋한 감정이 여름 들판과 소나기라는 배경 속에서 그려져 있다. 한 소년과 소녀의 풋풋한 감정이 여름
들판과 소나기라는 배경 속에서 그려져 있다. 한 소년과 소녀
